In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("flipkart_mobile_phones.csv")

In [5]:
df = df.drop("Unnamed: 0", axis = 1)

In [14]:
df["Sale & Actual Price"][0].split(df["Offer"][0])[0].split("₹")[1].replace(",", "")

'6999'

In [19]:
df.iloc[219]

Product Name            Samsung Galaxy A06 5G (Light Green, 64 GB)
Sale & Actual Price                           Price: Not Available
Raview & Rattings                         338 Ratings & 47 Reviews
Offer                                                     No Offer
Camera                   50MP + 50MP | 2MP + 2MP Dual Front Camera
Memory                 4 GB RAM | 64 GB ROM | Expandable Upto 1 TB
Battery                                           5000 mAh Battery
Display                            17.02 cm (6.7 inch) HD+ Display
Ratting Number                                                 4.2
Name: 219, dtype: object

In [21]:
sale_price = []
actual_price = []

for i in range(df.shape[0]):
    try:
        a = df["Sale & Actual Price"][i].split(df["Offer"][i])[0].split("₹")[1]
        sale_price.append(int(a.replace(",", "")))

        try:
            b = df["Sale & Actual Price"][i].split(df["Offer"][i])[0].split("₹")[2]
        except IndexError:
            b = a

        actual_price.append(int(b.replace(",", "")))
    except IndexError:
        sale_price.append(df["Sale & Actual Price"][i])
        actual_price.append(df["Sale & Actual Price"][i])


In [22]:
df["Sale Price"] = sale_price
df["Actual Price"] = actual_price

In [25]:
df = df.drop("Sale & Actual Price", axis = 1)

In [29]:
splitted_cols = df['Raview & Rattings'].str.split("&", expand=True)

In [35]:
df["No of Reviews"] = splitted_cols[0].str.replace(",", "").str.extract(r"(\d+)").astype(int)
df["No of Rattings"] = splitted_cols[1].str.replace(",", "").str.extract(r"(\d+)").astype(int)

In [38]:
final_df = df.drop("Raview & Rattings", axis = 1)

In [39]:
final_df.to_csv("processed_flipkart_mobile_phones.csv")

In [40]:
final_df.shape

(240, 11)

# Store into MongoDB

In [47]:
from pymongo import MongoClient

In [43]:
final_df

,Product Name,Offer,Camera,Memory,Battery,Display,Ratting Number,Sale Price,Actual Price,No of Reviews,No of Rattings
0,"POCO C71 (Cool Blue, 128 GB)",30% off,32MP Rear Camera | 8MP Front Camera,6 GB RAM | 128 GB ROM | Expandable Upto 2 TB,5200 mAh Battery,17.48 cm (6.88 inch) HD+ Display,4.1,6999,9999,5334,324
1,"Samsung Galaxy F05 (Twilight Blue, 64 GB)",35% off,50MP + 2MP | 8MP Front Camera,4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,5000 mAh Battery,17.12 cm (6.74 inch) HD+ Display,4.2,6499,9999,55419,3257
2,"REDMI A3X (Midnight Black, 64 GB)",40% off,8MP Rear Camera | 5MP Front Camera,3 GB RAM | 64 GB ROM | Expandable Upto 1 TB,5000 mAh Battery,17.04 cm (6.71 inch) HD+ Display,4.1,5999,9999,7906,355
3,"MOTOROLA g35 5G (Guava Red, 128 GB)",20% off,50MP + 8MP | 16MP Front Camera,4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,5000 mAh Battery,17.07 cm (6.72 inch) Full HD+ Display,4.2,9999,12499,56851,4373
4,"vivo T4x 5G (Marine Blue, 128 GB)",22% off,50MP + 2MP | 8MP Front Camera,6 GB RAM | 128 GB ROM,6500 mAh Battery,17.07 cm (6.72 inch) Display,4.4,13999,17999,73091,4034
...,...,...,...,...,...,...,...,...,...,...,...
235,"realme 14 Pro 5G (Pearl White, 256 GB)",10% off,50MP + 2MP | 16MP Front Camera,8 GB RAM | 256 GB ROM,6000 mAh Battery,17.2 cm (6.77 inch) Full HD+ Display,4.4,26999,29999,8530,732
236,"realme 14 Pro 5G (Jaipur Pink, 128 GB)",10% off,50MP + 2MP | 16MP Front Camera,8 GB RAM | 128 GB ROM,6000 mAh Battery,17.2 cm (6.77 inch) Full HD+ Display,4.4,24999,27999,8530,732
237,"Google Pixel 9A (Porcelain, 256 GB)",No Offer,48MP + 12MP | 12MP Front Camera,8 GB RAM | 256 GB ROM,5100 mAh Battery,15.96 cm (6.285 inch) Full HD+ Display,4.5,49999,49999,1304,147
238,Samsung Guru Music 2 B310ED,47% off,800 mAh Battery,32 MB RAM | 32 MB ROM,No Warranty,4.57 cm (1.8 inch) Display,3.2,990,1899,12843,1016


In [48]:
client = MongoClient('mongodb://localhost:27017/')

In [49]:
db = client["flipkart_database"]
collection = db["mobile_phones"]

In [51]:
data_dict = final_df.to_dict("records")

In [53]:
collection.insert_many(data_dict)
print("Dataframe inserted")

Dataframe inserted
